In [1]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("event_ods") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/22 04:40:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/22 04:40:56 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
# get all user from db, and create an rdd
from datetime import datetime
date_string = datetime.today().strftime('%Y-%m-%d')
cur = mydb.cursor()
cur.execute(f"SELECT tracking_events.*,{date_string} as date from tracking_events")
result = cur.fetchall()
all_event = sc.parallelize(result)
print(all_event.take(1))

[(1, 182, 78, 'view', datetime.datetime(2023, 4, 22, 0, 0), None, None, 1997)]


In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, DateType
schema = StructType([
    StructField("event_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("event_type", StringType(), True),
    StructField("timestamp", DateType(), True),
    StructField("page_depth", IntegerType(), True),
    StructField("current_url", StringType(), True),
    StructField("date", StringType(), True)
])
event_df = all_event.toDF(schema)
event_df.show()

+--------+-------+-------+----------------+----------+----------+-----------+----+
|event_id|user_id|item_id|      event_type| timestamp|page_depth|current_url|date|
+--------+-------+-------+----------------+----------+----------+-----------+----+
|       1|    182|     78|            view|2023-04-22|      null|       null|1997|
|       2|    182|    135|            view|2023-04-22|      null|       null|1997|
|       3|    182|    203|           click|2023-04-22|      null|       null|1997|
|       4|    182|    298|            view|2023-04-22|      null|       null|1997|
|       5|    182|    364|       long_view|2023-04-22|      null|       null|1997|
|       6|    182|    421|add_to_favorites|2023-04-22|      null|       null|1997|
|       7|    182|    123|            view|2023-04-22|      null|       null|1997|
|       8|    182|    257|           click|2023-04-22|      null|       null|1997|
|       9|    182|    356|            view|2023-04-22|      null|       null|1997|
|   

In [5]:
spark.sql("drop table if exists event_ods").show()

++
||
++
++



In [6]:
event_df.write.mode("overwrite").saveAsTable("event_ods")

23/04/22 04:41:18 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [7]:
spark.sql("show tables").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|      event_ods|      false|
| default|       item_ods|      false|
| default|   item_tag_ods|      false|
| default|item_word_count|      false|
| default|  item_word_idf|      false|
| default|   item_word_tf|      false|
| default|item_word_tfidf|      false|
| default|        tag_ods|      false|
| default|           test|      false|
| default|          test2|      false|
| default|       user_ods|      false|
| default|   user_tag_ods|      false|
+--------+---------------+-----------+



In [8]:
spark.stop()
mydb.close()